In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from PIL import Image, ImageEnhance
import urllib.request
from io import BytesIO
import numpy as np

BASE_URL = "https://letterboxd.com"

In [3]:
from main import get_user_diary_info

def generate_topster(user):
    # Generate topster for user

    user_df = get_user_diary_info(user)

    c_names = ["month", "day", "film", "released", "rating", "like", "rewatch", "review", "edityou", "film_url"]  # , "img_url", "small_img_url"]

    for i in range(len(c_names)):
        user_df.rename(columns={ user_df.columns[i]: c_names[i] }, inplace=True)

    # Sort by rating
    user_df = user_df.sort_values("rating", ascending=False).drop_duplicates('film_url', keep="first")
    return user_df

In [10]:
x = generate_topster("trschwab")

https://letterboxd.com/trschwab/films/diary/


/Users/troyschwab/Documents/github_letterboxd/letterboxd/movie_site/src/main.py:173: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 173 of the file /Users/troyschwab/Documents/github_letterboxd/letterboxd/movie_site/src/main.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(r.content)
/Users/troyschwab/Documents/github_letterboxd/letterboxd/movie_site/src/main.py:173: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it 

In [11]:
x

,month,day,film,released,rating,like,rewatch,review,edityou,film_url
31,"(, None)","(10, /trschwab/films/diary/for/2022/06/10/)","(Philip Glass: Akhnaten, /trschwab/film/philip...","(2019, None)","(× ★★★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/philip-glass-akhna...
18,"(, None)","(18, /trschwab/films/diary/for/2022/09/18/)","(Where Is the Friend's House?, /trschwab/film/...","(1987, None)","(× ★★★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/where-is-the-frien...
17,"(, None)","(25, /trschwab/films/diary/for/2021/04/25/)","(Superbad, /trschwab/film/superbad/1/)","(2007, None)","(× ★★★★★, #)","(, None)","(, None)","(Read prior review, /trschwab/film/superbad/1/)","(Edit this entry, #)",https://letterboxd.com/film/superbad/
23,"(, None)","(20, /trschwab/films/diary/for/2023/06/20/)","(Chungking Express, /trschwab/film/chungking-e...","(1994, None)","(× ★★★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/chungking-exp...","(Edit this entry, #)",https://letterboxd.com/film/chungking-express/
41,"(Mar 2022, /trschwab/films/diary/for/2022/03/)","(13, /trschwab/films/diary/for/2022/03/13/)","(Wet Hot American Summer, /trschwab/film/wet-h...","(2001, None)","(× ★★★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/wet-hot-ameri...","(Edit this entry, #)",https://letterboxd.com/film/wet-hot-american-s...
...,...,...,...,...,...,...,...,...,...,...
5,"(, None)","(24, /trschwab/films/diary/for/2022/11/24/)","(The Holiday, /trschwab/film/the-holiday/)","(2006, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/the-holiday/
28,"(, None)","(21, /trschwab/films/diary/for/2023/05/21/)","(Pierrot le Fou, /trschwab/film/pierrot-le-fou/)","(1965, None)","(×, #)","(, None)","(, None)","(Read the review, /trschwab/film/pierrot-le-fou/)","(Edit this entry, #)",https://letterboxd.com/film/pierrot-le-fou/
18,"(, None)","(11, /trschwab/films/diary/for/2021/04/11/)","(National Lampoon's Christmas Vacation, /trsch...","(1989, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/national-lampoons-...
27,"(, None)","(26, /trschwab/films/diary/for/2023/05/26/)","(Justice League: The Flashpoint Paradox, /trsc...","(2013, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/justice-league-the...


In [4]:
def post_user_into(user):
    user_df = get_user_diary_info(user)

    c_names = ["month", "day", "film", "released", "rating", "like", "rewatch", "review", "edityou", "film_url"]  # , "img_url", "small_img_url"]

    for i in range(len(c_names)):
        user_df.rename(columns={ user_df.columns[i]: c_names[i] }, inplace=True)

    user_df.to_csv("expected.csv")
    # explode tuple values into new cols:

    user_df[["month", "month_none"]] = pd.DataFrame(user_df['month'].tolist(), index=user_df.index)
    user_df[["day", "day_none"]] = pd.DataFrame(user_df['day'].tolist(), index=user_df.index)
    user_df[["film", "film_link"]] = pd.DataFrame(user_df['film'].tolist(), index=user_df.index)
    user_df[["released", "released_none"]] = pd.DataFrame(user_df['released'].tolist(), index=user_df.index)
    user_df[["rating", "rating_none"]] = pd.DataFrame(user_df['rating'].tolist(), index=user_df.index)
    user_df[["review", "review_link"]] = pd.DataFrame(user_df['review'].tolist(), index=user_df.index)

    # Convert to dict for iterative processing.. TODO fix this
    dict_df = user_df.to_dict('records')

    for count, item in enumerate(dict_df):
        if item["month"] == '':
            item["month"] = dict_df[count-1]["month"]

    dated_df = pd.DataFrame(dict_df)
    dated_df[["month", "year"]] = dated_df['month'].str.split(' ', expand=True)

    dated_df_dict = dated_df.to_dict('records')

    # Issue posts
    for item in dated_df_dict:
        data = {
        "name": user, 
        "day": item["day"],
        "month": item["month"],
        "year": item["year"],
        "film": item["film"],
        "released": item["released"],
        "rating":  item["rating"],
        "review_link": item["review_link"],
        "film_link" : item["film_link"]
        }

        # r = requests.post("http://127.0.0.1:8000/endpoint/hydrated_data_post/", data=data)

    return dated_df

In [14]:
post_user_into(x)

https://letterboxd.com/                                             month  ...                                           film_url
31                                        (, None)  ...  https://letterboxd.com/film/philip-glass-akhna...
18                                        (, None)  ...  https://letterboxd.com/film/where-is-the-frien...
17                                        (, None)  ...              https://letterboxd.com/film/superbad/
23                                        (, None)  ...     https://letterboxd.com/film/chungking-express/
41  (Mar 2022, /trschwab/films/diary/for/2022/03/)  ...  https://letterboxd.com/film/wet-hot-american-s...
..                                             ...  ...                                                ...
5                                         (, None)  ...           https://letterboxd.com/film/the-holiday/
28                                        (, None)  ...        https://letterboxd.com/film/pierrot-le-fou/
18            

/Users/troyschwab/Documents/github_letterboxd/letterboxd/movie_site/src/main.py:173: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 173 of the file /Users/troyschwab/Documents/github_letterboxd/letterboxd/movie_site/src/main.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(r.content)


ValueError: No tables found

In [5]:
# Let's take a step back and try to get a dataframe of user diary info but have it be a bit more detailed than the 
# default 
def post_user_into(user):
    user_df = get_user_diary_info(user)
    return user_df

In [17]:
post_user_into("trschwab")

https://letterboxd.com/trschwab/films/diary/


/Users/troyschwab/Documents/github_letterboxd/letterboxd/movie_site/src/main.py:173: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 173 of the file /Users/troyschwab/Documents/github_letterboxd/letterboxd/movie_site/src/main.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(r.content)
/Users/troyschwab/Documents/github_letterboxd/letterboxd/movie_site/src/main.py:173: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it 

,"(Month, None)","(Day, None)","(Film, None)","(Released, None)","(Rating, None)","(Like, None)","(Rewatch, None)","(Review, None)","(EditYou, None)",film_url
0,"(Nov 2023, /trschwab/films/diary/for/2023/11/)","(26, /trschwab/films/diary/for/2023/11/26/)","(Oppenheimer, /trschwab/film/oppenheimer-2023/1/)","(2023, None)","(×, #)","(, None)","(, None)","(Read the review, /trschwab/film/oppenheimer-2...","(Edit this entry, #)",https://letterboxd.com/film/oppenheimer-2023/
1,"(, None)","(22, /trschwab/films/diary/for/2023/11/22/)","(Step Brothers, /trschwab/film/step-brothers/)","(2008, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/step-brothers/
2,"(, None)","(22, /trschwab/films/diary/for/2023/11/22/)","(The Impossible, /trschwab/film/the-impossible/)","(2012, None)","(×, #)","(, None)","(, None)","(Read the review, /trschwab/film/the-impossible/)","(Edit this entry, #)",https://letterboxd.com/film/the-impossible/
3,"(, None)","(19, /trschwab/films/diary/for/2023/11/19/)","(A Star Is Born, /trschwab/film/a-star-is-born...","(2018, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/a-star-is-born-2018/
4,"(, None)","(09, /trschwab/films/diary/for/2023/11/09/)","(Taste of Cherry, /trschwab/film/taste-of-cher...","(1997, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/taste-of-cherry/
...,...,...,...,...,...,...,...,...,...,...
25,"(, None)","(12, /trschwab/films/diary/for/2020/01/12/)","(The Wind Rises, /trschwab/film/the-wind-rises/)","(2013, None)","(× ★★★★½, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/the-wind-rises/
26,"(, None)","(11, /trschwab/films/diary/for/2020/01/11/)","(Badlands, /trschwab/film/badlands/)","(1973, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/badlands/
27,"(, None)","(09, /trschwab/films/diary/for/2020/01/09/)","(Chef, /trschwab/film/chef/)","(2014, None)","(× ★★, #)","(, None)","(, None)","(Read prior review, /trschwab/film/chef/)","(Edit this entry, #)",https://letterboxd.com/film/chef/
28,"(, None)","(07, /trschwab/films/diary/for/2020/01/07/)","(Wet Hot American Summer, /trschwab/film/wet-h...","(2001, None)","(× ★★★½, #)","(, None)","(, None)","(Read prior review, /trschwab/film/wet-hot-ame...","(Edit this entry, #)",https://letterboxd.com/film/wet-hot-american-s...


In [6]:

def get_user_diary_info(a_user):
    '''
    returns a DF of a user's diary information

    # TOOD need to be able to handle users that do not exist
    '''
    df_list = []
    # Get diary URL
    diary_url = get_diary(a_user)
    print(diary_url)
    while diary_url:
        # print(diary_url)
        r = requests.get(diary_url)
        soup = BeautifulSoup(r.content)

        # Extract the table as a df
        table = soup.find_all('table')
        df_list.append(pd.read_html(str(table), extract_links="all")[0])

        # Find out if there are any subsequent pages
        older_link = soup.find_all("a", text="Older")

        if older_link:
            new_page = older_link[0]['href']
            diary_url = f"{BASE_URL}{new_page}"
        else:
            diary_url = False
    return_df = pd.concat(df_list)

    return_df["film_url"] = return_df[('Film', None)].apply(lambda x: gen_film_url(x))

    return return_df

In [7]:

def get_diary(a_user):
    return f"{BASE_URL}/{a_user}/films/diary/"

In [26]:
a_user = "trschwab"
df_list = []
# Get diary URL
diary_url = get_diary(a_user)
print(diary_url)
i = 0
while diary_url:
    print(diary_url)
    r = requests.get(diary_url)
    soup = BeautifulSoup(r.content)
    f = open(f"demofile{i}.txt", "w")
    f.write(str(soup))
    f.close()

    # Extract the table as a df
    table = soup.find_all('table')
    df_list.append(pd.read_html(str(table), extract_links="all")[0])

    # Find out if there are any subsequent pages
    older_link = soup.find_all("a", text="Older")

    if older_link:
        new_page = older_link[0]['href']
        diary_url = f"{BASE_URL}{new_page}"
    else:
        diary_url = False
    i += 1

https://letterboxd.com/trschwab/films/diary/
https://letterboxd.com/trschwab/films/diary/
https://letterboxd.com/trschwab/films/diary/page/2/
https://letterboxd.com/trschwab/films/diary/page/3/
https://letterboxd.com/trschwab/films/diary/page/4/
https://letterboxd.com/trschwab/films/diary/page/5/
https://letterboxd.com/trschwab/films/diary/page/6/


In [8]:
# Let's try to get a dataframe of info about a given movie
goon = "https://letterboxd.com/film/goon/"

In [9]:
r = requests.get(goon)
soup = BeautifulSoup(r.content)

In [30]:
f = open(f"goon.txt", "w")
f.write(str(soup))
f.close()

In [10]:
def get_page(url):
	s = requests.session()
	r = s.get(url)
	soup = BeautifulSoup(r.text, "html.parser")
	return soup

In [11]:
def get_a_movie_info(url):
	soup = get_page(url)
	i = 0
	begin = 0
	end = 0
	for item in str(soup).split('\n'):
		if ("* <![CDATA[ */" in item):
			begin = i
		if ("/* ]]> */" in item):
			end = i + 1
		i += 1
	return [str(soup).split('\n')[begin:end], url]

In [13]:
test_info = get_a_movie_info(goon)

In [26]:
json.loads(test_info[0][1])

{'image': 'https://a.ltrbxd.com/resized/film-poster/6/1/9/2/8/61928-goon-0-230-0-345-crop.jpg?v=07c7938d9e',
 '@type': 'Movie',
 'director': [{'@type': 'Person',
   'name': 'Michael Dowse',
   'sameAs': '/director/michael-dowse/'}],
 'dateModified': '2023-07-09',
 'productionCompany': [{'@type': 'Organization',
   'name': 'Inferno Pictures',
   'sameAs': '/studio/inferno-pictures/'},
  {'@type': 'Organization',
   'name': 'Don Carmody Productions',
   'sameAs': '/studio/don-carmody-productions/'},
  {'@type': 'Organization',
   'name': 'Caramel Films',
   'sameAs': '/studio/caramel-films/'},
  {'@type': 'Organization',
   'name': 'No Trace Camping',
   'sameAs': '/studio/no-trace-camping/'},
  {'@type': 'Organization',
   'name': 'DragonCove Studios',
   'sameAs': '/studio/dragoncove-studios/'}],
 'releasedEvent': [{'@type': 'PublicationEvent', 'startDate': '2011'}],
 '@context': 'http://schema.org',
 'url': 'https://letterboxd.com/film/goon/',
 'actors': [{'@type': 'Person',
   'name'

In [24]:
movie_df = pd.json_normalize(json.loads(test_info[0][1]))

ValueError: All arrays must be of the same length

In [38]:
def remove_apos(s):
	''' takes in a string and removes all ' or "
	'''
	#try escaping instead..
	return str(s).replace("'","").replace('"',"")


In [39]:
def get_b_movie_info(url):
	soup = get_page(url)
	#title = str(soup.find_all(property="og:title"))
	desc = str(soup.find_all(property="og:description"))
	cover_image = str(soup.find_all(property="og:image"))
	#title = title.split('"')[1][:-7]
	desc = desc.split('"')[1]
	cover_image = cover_image.split('"')[1]
	#r_list = [remove_apos(title), remove_apos(desc), remove_apos(cover_image)]
	r_list = [remove_apos(desc), remove_apos(cover_image)]
	#print("b movie info")
	#print(r_list)
	return str(r_list)

In [40]:
get_b_movie_info(goon)

"['Doug Glatt, a slacker who discovers he has a talent for brawling is approached by a minor league hockey coach and invited to join the team as the &quot;muscle.&quot; Despite the fact that Glatt cant skate his best friend, Pat, convinces him to give it a shot, and Glatt becomes a hero to the team and their fans, until the leagues reigning goon becomes threatened by Glatts success and decides to even the score.', 'https://a.ltrbxd.com/resized/sm/upload/83/69/xi/b5/goon-1200-1200-675-675-crop-000000.jpg?v=3cc173ca18']"